# **Installing the Required Libraries**

### **`langchain`**
- LangChain is a framework used to build AI applications using large language models (LLMs).
- It helps connect models with tools, memory, prompts, and agents.

### **`langchain-google-genai`**
- This library helps you use Google's Generative AI models (like Gemini) directly inside LangChain.

### **`google-generativeai`**
- Official Google library to access Gemini models using API keys without LangChain.

### **`tiktoken`**
- Used for counting tokens (words/characters) so you don’t exceed LLM token limits (important for pricing and requests).

### **`faiss-cpu`**
- FAISS is a vector database used to store text embeddings and search relevant chunks quickly. Used in chatbots and RAG.

### **`huggingface_hub`**
- Allows downloading pre-trained models (like embedding models) from Hugging Face.

### **`langchain-community`**
- Contains community-supported integrations like FAISS, agents, tools, vectorstores, embeddings, etc.

### **`langgraph`**
- Optional – Used for creating complex AI workflows or multi-step pipelines.

In [ ]:
!pip install langchain langchain-google-genai google-generativeai tiktoken faiss-cpu huggingface_hub langchain-community langgraph

  Using cached google_ai_generativelanguage-0.7.0-py3-none-any.whl.metadata (10 kB)
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Using cached langchain_google_genai-2.1.12-py3-none-any.whl.metadata (7.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-google-genai
    Found existing installation: langchain-google-genai 2.1.12
    Uninstalling langchain-google-genai-2.1.12:
      Successfully uninstalled langchain

# **Insert API Key**

In [ ]:
from google.colab import userdata
# Used to store and access private values like API keys securely in Google Colab.
key = userdata.get('GOOGLE_API_KEY')

import google.generativeai as genai
# This lets you directly use Gemini models with your API key (without LangChain).
genai.configure(api_key= key)

#### **Check available Models in google gen ai library**

In [ ]:
available_models = genai.list_models()
list(available_models)

[Model(name='models/embedding-gecko-001',
       base_model_id='',
       version='001',
       display_name='Embedding Gecko',
       description='Obtain a distributed representation of a text.',
       input_token_limit=1024,
       output_token_limit=1,
       supported_generation_methods=['embedText', 'countTextTokens'],
       temperature=None,
       max_temperature=None,
       top_p=None,
       top_k=None),
 Model(name='models/gemini-2.5-pro-preview-03-25',
       base_model_id='',
       version='2.5-preview-03-25',
       display_name='Gemini 2.5 Pro Preview 03-25',
       description='Gemini 2.5 Pro Preview 03-25',
       input_token_limit=1048576,
       output_token_limit=65536,
       supported_generation_methods=['generateContent',
                                     'countTokens',
                                     'createCachedContent',
                                     'batchGenerateContent'],
       temperature=1.0,
       max_temperature=2.0,
       top_p=0.9

In [ ]:
# To get a model separately
# gen_ai_model = genai.GenerativeModel('gemini-2.0-flash')

# **Creating LLM**

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
# Used when you want to use Gemini inside LangChain instead of directly through the Google library.

llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.0-flash',
    google_api_key = key,
    temperature = 0,
    max_output_tokens = 512
)

# **Agentic AI : ReAct Pattern or Tool use Pattern**
- Reasoning + action alternately

In [ ]:
from langchain.schema import HumanMessage, AIMessage
"""
These are structured message formats used in LangChain to store:
- Messages from the user → HumanMessage
- Responses from the AI → AIMessage
This helps in chat history management, which is essential for multi-turn conversations.
"""
class Agent:
  def __init__(self):
    self.messages = [] #message history - maintain
    """
    Creates an empty list to store past conversation messages.
    """

  #dunder method
  def __call__(self,message):
    """
    This is a dunder method.
    It allows the object to be called like a function:
    - `agent("Hello")` is same as `agent.__call__("Hello")`
    """
    self.messages.append(HumanMessage(content=message))
    reponse = self.execute()
    self.messages.append(AIMessage(content=reponse))
    return reponse
    """
    What it does:
    - Stores the user input (HumanMessage)
    - Calls execute() to get AI response
    - Adds AI reply (AIMessage) to history
    - Returns the response

    This allows the agent to maintain memory across turns.
    """


  def execute(self):
    response = llm(self.messages)
    return response.content
    """
    - Calls the LLM (e.g., Gemini, GPT, etc.)
    - Sends the full conversation history
    - Returns only the text content
    """

In [ ]:
import re

action_re = re.compile(r'Action: \s*(\w+):\s*(.*?)(?:\n|$)',re.MULTILINE)

This regex is designed to extract structured information from text like:

- Action: Move: forward
- Action: Attack: dragon
- Action: Speak: Hello there

# **Write code for the type of agent we want**

In [ ]:
def calculate(expression):
    return eval(expression)

def convert_currency(data):
    """
    Expects input like: "<amount> <CURRENCY_CODE> to <TARGET_CURRENCY>"
    or a single pair like: "10 USD" (we'll assume convert TO INR if no target given).
    Examples:
      "10 USD to INR"
      "5 EUR to USD"
      "100 JPY"
    """
    try:
        parts = data.strip().split()
        # Accept either "AMOUNT SRC to TGT" or "AMOUNT SRC"
        if len(parts) == 2:
            amount = float(parts[0])
            src = parts[1].upper()
            tgt = "INR"
        elif len(parts) == 4 and parts[2].lower() == "to":
            amount = float(parts[0])
            src = parts[1].upper()
            tgt = parts[3].upper()
        else:
            return "Invalid input format. Use: '<amount> <SRC>' or '<amount> <SRC> to <TGT>'"

        # Static example rates (relative to 1 unit of the currency -> INR)
        # These are sample/static values. For production use, query a live API.
        inr_rates = {
            "USD": 83.12,
            "EUR": 89.45,
            "GBP": 104.30,
            "AUD": 55.60,
            "CAD": 60.25,
            "JPY": 0.56,
            "CNY": 11.48,
            "AED": 22.65,
            "SGD": 61.15,
            "CHF": 93.20,
            "HKD": 10.65,
            "NZD": 50.40,
            "KRW": 0.064,
            "SEK": 8.20,
            "NOK": 8.10,
            "DKK": 11.95,
            "RUB": 0.88,
            "BRL": 16.20,
            "ZAR": 4.35,
            "INR": 1.0  # base
        }

        def to_inr(value, src_code):
            if src_code not in inr_rates:
                return None
            return value * inr_rates[src_code]

        def from_inr(value_inr, tgt_code):
            if tgt_code not in inr_rates:
                return None
            return value_inr / inr_rates[tgt_code]

        # Convert src -> INR
        src_to_inr = to_inr(amount, src)
        if src_to_inr is None:
            return f"Source currency '{src}' not supported."

        # If target is INR, we are done
        if tgt == "INR":
            return f"{amount} {src} = {src_to_inr:.2f} INR"

        # Convert INR -> target
        converted = from_inr(src_to_inr, tgt)
        if converted is None:
            return f"Target currency '{tgt}' not supported."

        return f"{amount} {src} = {converted:.6f} {tgt} (≈ {src_to_inr:.2f} INR)"
    except Exception as e:
        return f"Error parsing input: {str(e)}. Use '<amount> <SRC>' or '<amount> <SRC> to <TGT>'."

known_actions = {
    "calculate": calculate,
    "convert_currency": convert_currency
}

In [ ]:
def run_query(prompt, max_turns = 10):  # Increased max_turns
    agent = Agent()
    """
    Starts an agent
    """
    next_prompt = prompt

    for turn in range(max_turns):
        print(f"Turn {turn + 1}:")
        response = agent(next_prompt)
        print(f"Assistant: \n{response}\n")

        """
        - Loops up to max_turns
        - Sends prompt to the agent
        - Prints the assistant’s response
        """

        # Check for action lines in the response
        actions = action_re.findall(response)

        if actions:
            action, action_input = actions[0]
            print(f"acrtion is :{actions}")
            action_input = action_input.strip()

            if action not in known_actions:
                raise ValueError(f"Unknown action: {action}")

            observation = known_actions[action](action_input)
            print(f"Observation: {observation}\n")
            next_prompt = f"Observation: {observation}"

            """
            - If the LLM outputs something like "Action: X",
            - The agent calls a real function from known_actions
            - Sends the result back as "Observation"
            """
        else:
            # No more actions found, check if we have a final answer
            if "Answer:" in response:
                print("Final answer provided!")
                break
            else:
                print("No action found and no final answer. Stopping.")
                break
        """
        Stops when the AI writes "Answer: ..."
        Otherwise ends after max turns
        """

In [ ]:
instructions_and_question = """
You are a helpful assistant following a ReAct reasoning pattern.

For each step:
1. Use "Thought:" to think step by step about the problem
2. If you need to perform a conversion or calculation, use 'Action: <action_name>: <input>' on a new line
3. After each action, I will provide an 'Observation:' with the result
4. Continue thinking and acting until you can provide a final 'Answer:'

Available actions:
- calculate: perform a Python calculation (e.g., calculate: 5 + 3)
- convert_currency: Convert currencies. Use inputs like "10 USD", "10 USD to INR", or "5 EUR to USD".

Important: Only perform ONE action at a time, then wait for the observation.

Question: I have 10 USD and 20 EUR. What is their combined value in INR?
"""


In [ ]:
run_query(instructions_and_question)

Turn 1:
Assistant: 
Thought: Okay, I need to convert both USD and EUR to INR and then add them together. First, I'll convert 10 USD to INR.
Action: convert_currency: 10 USD to INR

acrtion is :[('convert_currency', '10 USD to INR')]
Observation: 10.0 USD = 831.20 INR

Turn 2:
Assistant: 
Thought: Now I need to convert 20 EUR to INR.
Action: convert_currency: 20 EUR to INR

acrtion is :[('convert_currency', '20 EUR to INR')]
Observation: 20.0 EUR = 1789.00 INR

Turn 3:
Assistant: 
Thought: Now I need to add the two INR values together to get the total.
Action: calculate: 831.20 + 1789.00

acrtion is :[('calculate', '831.20 + 1789.00')]
Observation: 2620.2

Turn 4:
Assistant: 
Thought: I have converted both currencies to INR and added them.
Answer: 10 USD and 20 EUR is equivalent to 2620.2 INR.

Final answer provided!
